# 1. Load Data

In [1]:
import json
import pandas as pd
import numpy as np
import nltk

In [2]:
data = []
with open('./data/tweets_DM.json', 'r') as f:
    for line in f:
        try:
            data.append(json.loads(line))  # Safeguard against malformed JSON
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

f.close()

In [3]:
emotion_list = pd.read_csv('./data/emotion.csv')
data_identification = pd.read_csv('./data/data_identification.csv')

In [4]:
df = pd.DataFrame(data)

# Extract '_source' and validate structure
if '_source' not in df.columns:
    raise KeyError("'_source' column not found in the data")

_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})

# Ensure tweet_id is of a consistent type
df['tweet_id'] = df['tweet_id'].astype(str)

# Validate and prepare `data_identification`
data_identification['tweet_id'] = data_identification['tweet_id'].astype(str)
df = df.merge(data_identification, on='tweet_id', how='left')

train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']

In [5]:
train_data = train_data.merge(emotion_list, on='tweet_id', how='left')
train_data.head()

,tweet_id,hashtags,text,identification,emotion
0,0x376b20,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,0x2d5350,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,0x1cd5b0,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,0x1d755c,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,0x2c91a8,[],Still waiting on those supplies Liscus. <LH>,train,anticipation


In [6]:
test_data.head()

,tweet_id,hashtags,text,identification
2,0x28b412,[bibleverse],"Confident of your obedience, I write to you, k...",test
4,0x2de201,[],"""Trust is not the same as faith. A friend is s...",test
9,0x218443,"[materialism, money, possessions]",When do you have enough ? When are you satisfi...,test
30,0x2939d5,"[GodsPlan, GodsWork]","God woke you up, now chase the day #GodsPlan #...",test
33,0x26289a,[],"In these tough times, who do YOU turn to as yo...",test


In [7]:
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

In [8]:
# shuffle dataset
train_data = train_data.sample(frac=1)
test_data = test_data.sample(frac=1)

print("Shape of Training df: ", train_data.shape)
print("Shape of Testing df: ", test_data.shape)
train_data.head()

Shape of Training df:  (1449182, 5)
Shape of Testing df:  (411972, 4)


,tweet_id,hashtags,text,identification,emotion
429211,0x21216c,[faith],A new test requires a new level of #faith...,train,anticipation
671936,0x363ca7,"[iPhoneX, WatchingOrignialiPhonePresentation]",Does any other @Apple lover really miss Steve ...,train,sadness
487977,0x24bd66,[],What are you <LH> for? Me? My family (especial...,train,trust
48295,0x37c902,[Cutie],#Cutie <LH> <LH> <LH> @MayaRyanN10😍😘😚☺️😗😙😚☺️sh...,train,anticipation
794952,0x2ccacb,[Sad],@KathieLGifford Kathie have another glass of w...,train,sadness


In [9]:
test_data.head()

,tweet_id,hashtags,text,identification
1301736,0x26ef3d,"[plank, Bringyourownmusic, awesomegig, awesome...",Day 4 - A one-minute #plank every two hours. #...,test
1415238,0x2dcc15,[lymesucks],& I was never allergic to anything & now i hav...,test
988609,0x2ea4de,[],It's crazy how I'm the one with the pretty gir...,test
877281,0x2f7f57,[],@captain_drew410 @Gibson05141819 I didn't take...,test
938902,0x2a181f,"[heeltexas, ihate45]",@realDonaldTrump Can you stop pretending to be...,test


In [10]:
train_data_sample = train_data.sample(frac=0.3, random_state=42)

In [11]:
y_train_alter = train_data_sample['emotion']
y_train_data = pd.DataFrame(y_train_alter)
X_train_data = train_data_sample.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)
ans_data = test_data.drop(['tweet_id', 'identification', 'hashtags'], axis=1)

In [12]:
y_train_data.head()

,emotion
961482,sadness
1192387,anticipation
1211362,joy
997538,trust
1403990,anger


In [13]:
X_train_data.head()

,text
961482,@MostRequestLive Please Play <LH> by @FifthHar...
1192387,The only way to #improve how #clearly you hear...
1211362,Finally it's friday! i get to chilax now!😌😌 <L...
997538,Someone at the @Mcr_Storm <LH> game tweet us s...
1403990,I hate how Windows freezes and forces you to h...


In [14]:
ans_data.head()

,text
1301736,Day 4 - A one-minute #plank every two hours. #...
1415238,& I was never allergic to anything & now i hav...
988609,It's crazy how I'm the one with the pretty gir...
877281,@captain_drew410 @Gibson05141819 I didn't take...
938902,@realDonaldTrump Can you stop pretending to be...


# 2. Deep learning

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_data, y_train_data, test_size=0.2, random_state=42)

In [16]:
import keras

### 2.1 BOW is still needed

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\Daisy
[nltk_data]     Liu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [18]:
# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize,
                         token_pattern=None)

In [19]:
# apply analyzer to training data
BOW_500.fit(train_data['text'])

CountVectorizer(max_features=500, token_pattern=None,
                tokenizer=<function word_tokenize at 0x00000199A7D3D080>)

### 2.2 Prepare the data(X, y)

In [20]:
y_train.head()

,emotion
931183,anticipation
977540,anticipation
23221,disgust
132951,anger
533987,sadness


In [21]:
X_test.head()

,text
876882,<LH> my girl @KellyKhumaloZA❤️❤️❤️happy can't ...
215055,@thegroush @Joey7Barton You should always bet ...
1444730,Yasss @MATTHARDYBRAND is finally <LH> or is it...
138396,Hay @MINIUK did you get my DM you asked for! A...
54751,@IamAnupK a msg from biggest fan of #captain <...


In [22]:
# standardize name (X, y)
X_trainv2 = BOW_500.transform(X_train['text'])
y_trainv2 = y_train['emotion']

X_testv2 = BOW_500.transform(X_test['text'])
y_testv2 = y_test['emotion']

## check dimension is a good habbit
print('X_train.shape: ', X_trainv2.shape)
print('y_train.shape: ', y_trainv2.shape)
print('X_test.shape: ', X_testv2.shape)
print('y_test.shape: ', y_testv2.shape)

X_train.shape:  (347804, 500)
y_train.shape:  (347804,)
X_test.shape:  (86951, 500)
y_test.shape:  (86951,)


In [23]:
ans_data.head()

,text
1301736,Day 4 - A one-minute #plank every two hours. #...
1415238,& I was never allergic to anything & now i hav...
988609,It's crazy how I'm the one with the pretty gir...
877281,@captain_drew410 @Gibson05141819 I didn't take...
938902,@realDonaldTrump Can you stop pretending to be...


In [24]:
ans_datav2 = BOW_500.transform(ans_data['text'])

In [25]:
ans_datav2.shape

(411972, 500)

### 2.3 Deal with categorical label(y)

In [26]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_trainv2)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_trainv2[0:4])
print('\ny_train.shape: ', y_trainv2.shape)
print('y_test.shape: ', y_testv2.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_trainv2 = label_encode(label_encoder, y_trainv2)
y_testv2 = label_encode(label_encoder, y_testv2)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_trainv2[0:4])
print('\ny_train.shape: ', y_trainv2.shape)
print('y_test.shape: ', y_testv2.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 931183    anticipation
977540    anticipation
23221          disgust
132951           anger
Name: emotion, dtype: object

y_train.shape:  (347804,)
y_test.shape:  (86951,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]

y_train.shape:  (347804, 8)
y_test.shape:  (86951, 8)


### 2.4 Build model

In [27]:
# I/O check
input_shape = X_trainv2.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [28]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 8)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,744 (143.53 KB)

 Trainable params: 36,744 (143.53 KB)

 Non-trainable params: 0 (0.00 B)

### 2.5 Train

In [29]:
from keras.callbacks import CSVLogger

# csv_logger = CSVLogger('/content/drive/My Drive/NTHU/DM-Lab2-HW/logs/training_log.csv')

# training setting
epochs = 40
batch_size = 512

# training!
history = model.fit(X_trainv2, y_trainv2,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data = (X_testv2, y_testv2))
print('training finish')

Epoch 1/40


C:\Users\Daisy Liu\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


680/680 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.4106 - loss: 1.6123 - val_accuracy: 0.4684 - val_loss: 1.4626
Epoch 2/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4731 - loss: 1.4471 - val_accuracy: 0.4783 - val_loss: 1.4345
Epoch 3/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4849 - loss: 1.4100 - val_accuracy: 0.4833 - val_loss: 1.4213
Epoch 4/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4912 - loss: 1.3933 - val_accuracy: 0.4853 - val_loss: 1.4139
Epoch 5/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4969 - loss: 1.3782 - val_accuracy: 0.4862 - val_loss: 1.4115
Epoch 6/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4996 - loss: 1.3716 - val_accuracy: 0.4883 - val_loss: 1.4074
Epoch 7/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5028 - loss: 1.3628 - val_accuracy: 0.4866 - val_loss: 1.4069
Epoch 8/40
680/680 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5061 - loss: 1.3542 - val_accuracy: 0.4892 - val_

### 2.6 Predict data

In [30]:
pred_X_test = model.predict(X_testv2, batch_size=128)
pred_X_test[:5]

680/680 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[3.3443570e-04, 1.3774487e-01, 8.7140867e-04, 3.1081237e-02,
        5.5267721e-01, 4.3851319e-03, 2.4453454e-02, 2.4845220e-01],
       [2.4110463e-02, 1.6987316e-02, 2.7212620e-01, 9.7434586e-03,
        2.4894217e-01, 2.0885427e-01, 1.9791869e-02, 1.9944428e-01],
       [4.4198367e-03, 1.4131893e-01, 3.7889674e-02, 1.3347687e-02,
        5.0205648e-01, 2.0193119e-01, 7.4487943e-03, 9.1587380e-02],
       [1.1503821e-02, 1.7009306e-01, 9.3993582e-03, 3.3802520e-02,
        4.7497466e-01, 3.2848861e-02, 6.1779741e-02, 2.0559798e-01],
       [1.0074459e-02, 5.9016723e-02, 4.0224932e-02, 1.6736675e-02,
        5.0105011e-01, 7.0745707e-02, 3.8683467e-02, 2.6346788e-01]],
      dtype=float32)

In [31]:
pred_X_test = label_decode(label_encoder, pred_X_test)
pred_X_test[:5]

array(['joy', 'disgust', 'joy', 'joy', 'joy'], dtype=object)

In [32]:
from sklearn.metrics import accuracy_score
#Accuracy
print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_testv2), pred_X_test), 2)))

testing accuracy: 0.48


In [33]:
## predict
pred_result = model.predict(ans_datav2, batch_size=64)
pred_result[:5]

6438/6438 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step


array([[0.03435718, 0.04779501, 0.06556747, 0.03090484, 0.5461336 ,
        0.05032621, 0.01131507, 0.21360064],
       [0.01342897, 0.05141359, 0.2510993 , 0.01829807, 0.25891966,
        0.17758341, 0.02898435, 0.20027263],
       [0.0235979 , 0.00514851, 0.1013723 , 0.32822978, 0.13494389,
        0.09550518, 0.16441798, 0.14678441],
       [0.02585692, 0.06056243, 0.10118613, 0.04569026, 0.14424786,
        0.4680472 , 0.04166508, 0.11274406],
       [0.01875504, 0.02806245, 0.34473905, 0.00939836, 0.02173029,
        0.48617205, 0.06465231, 0.0264904 ]], dtype=float32)

In [34]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['joy', 'joy', 'fear', 'sadness', 'sadness'], dtype=object)

In [35]:
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': pred_result,
})

In [40]:
submission.to_csv('./submission.csv', index=False)